<a href="https://colab.research.google.com/github/emyesme/CalcificationDetection/blob/Emily/Pipeline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries and Data Setup

## Import libraries

In [21]:
# just once to install opencv
#!pip install opencv-python
#!pip install matplotlib
#!pip install numpy
#!pip install google-colab

#!pip install PyWavelets
#!pip install image_dehazer
!pip install -U scikit-image
#!pip install fastprogress
from fastprogress import master_bar, progress_bar

# import opencv
import cv2
# import numpy
import numpy as np
import math
# features module from scikit-image
from skimage import feature
import itertools
#import show special for google colab
from google.colab.patches import cv2_imshow
#import plt for display
import matplotlib.pyplot as plt
# wavelet
import pywt
# combination of lists
import itertools

## Drive Setup

In [22]:
from google.colab import drive
drive.mount('/content/drive') # This will prompt for authorization.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We put a shortcut in our drive to the image processing folder for this.

In [23]:
import os 
DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')

## Data Setup

In [24]:
#go into de directory of the images

# this have 3 outputs root directory, the folders in the path and the files in the path.
# we ignore _ the two first because we are not interested in those
_, _, images = next(os.walk(os.path.join(DATA_DIR,'images')))
_, _, breastMasks = next(os.walk(os.path.join(DATA_DIR,'masks')))
_, _, groundTruths = next(os.walk(os.path.join(DATA_DIR, 'groundtruths')))

images.sort()
breastMasks.sort()
groundTruths.sort()

# read numbers of normal images
normals = []
with open(os.path.join(DATA_DIR,'normals.txt')) as f:
    for line in f:
        normals.append(line[:-1])

## Google Colab Like a Pro

In [25]:
# https://medium.com/@robertbracco1/configuring-google-colab-like-a-pro-d61c253f7573#a642
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

# Preprocessing

## DeHazing Using Dark Channel Prior and Guided Filter

Dehazing method proposed by the professor (also used in his paper)

Taken from here:
https://github.com/He-Zhang/image_dehaze
Info on readme.md of the repo

### Dark Channel

Here goes the theory behind this function 

In [26]:
# Here goes inputs --> output types
def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

### AtmLight

In [27]:
# Possibly change to grayscale would be nice
def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx;
    return A

### TransmissionEstimate

In [28]:
def TransmissionEstimate(im,A,sz):
    omega = 0.95;# the closer to 1 the stronger the darkenning
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

### GuidedFilter

In [29]:
def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

### TransmissionRefine

In [30]:
def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

### Recover

In [31]:
def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]
    return res

### deHazingDarkChannelPriorPy

In [32]:
def deHazingDarkChannelPriorPy(matrix, mask):

    I = matrix.astype(np.float64)/255
 
    dark = DarkChannel(I,15)
    A = AtmLight(I,dark)
    te = TransmissionEstimate(I,A,15)
    t = TransmissionRefine(matrix,te)
    J = Recover(I,t,A,0.1)
    preprocessed = J
    return preprocessed

# image = cv2.imread(DATA_DIR+"/images/53582422_3f0db31711fc9795_MG_R_ML_ANON.tif")
# dark, t, matrix, J = deHazingDarkChannelPriorPy(image, image)

## Image Dilation

In [33]:
def imgDilation(matrix):
  kernel = np.ones((9,9), np.uint8)
  img_dilation = cv2.dilate(matrix, kernel, iterations=3)
  return img_dilation

## Other Tried Methods

**CLAHE (adaptive histogram equalization)**
  * CLAHE + dehazing = bad results, black image 
  * points less visible with CLAHE

**Linear stretching**
  * still missing linear streching
  * The code goes on forever (high computational cost)



# Candidate Extraction

##Hessian-Matrix-Based Analysis

Hessian-matrix-based analysis or difference of gaussians (DoH) blob detection from skimage:

https://scikit-image.org/docs/stable/api/skimage.feature.html?highlight=local%20binary%20pattern#skimage.feature.blob_doh

In [34]:
def candidateExtraction(matrix, mask):

  from skimage import feature
  
  # returns x,y,sigma of the blob
  blobs = feature.blob_doh(matrix,
                           min_sigma=1,
                           max_sigma=30,
                           num_sigma=10,
                           # The absolute lower bound for scale space maxima.
                           # Local maxima smaller than threshold are ignored.
                           # Reduce this to detect blobs with lower intensities.
                           # If threshold_rel is also specified, whichever threshold is larger will be used.
                           # If None, threshold_rel is used instead.
                           threshold=0.005,
                           # lower more sensible, more false positives bad also tinier calcifications detected
                           overlap=0.5,
                           log_scale=False,
                           threshold_rel=None
                           )
  # taken from the documentation
  # ...The downside is that this method can’t be used for detecting blobs of radius less than 3px
  # due to the box filters used in the approximation of Hessian Determinant.
  result = blobs
  return result

# Feature Extraction

In [35]:
# Get Ground Truth for each patch
def patchGroundTruth(candidate, groundTruth):

  left = int((candidate[0] - candidate[2]) if ((candidate[0] - candidate[2]) > 0) else 0)
  right = int((candidate[0] + candidate[2]) if ((candidate[0] + candidate[2]) < groundTruth.shape[0]) else groundTruth.shape[0])
  top = int((candidate[1] - candidate[2]) if ((candidate[1] - candidate[2]) > 0) else 0)
  bottom = int((candidate[1] + candidate[2]) if ((candidate[1] + candidate[2]) < groundTruth.shape[1]) else groundTruth.shape[1])

  truePatch = groundTruth[left : right,
                          top  : bottom]
  sum = np.sum(truePatch)
  if sum > 0:
    return str(1)
  else:
    return str(0)

Video tutorial on how to use GLCM:

* https://www.youtube.com/watch?v=5x-CIHRmMNY

Documentation and example on GLCM:

* https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_glcm.html

Paper:

*  https://ijcrr.com/uploads/3454_pdf.pdf

GLCM Properties documentation:

* https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.graycoprops

GLCM Gray co-matrix documentation:

* https://stackoverflow.com/questions/54512617/creating-gray-level-co-occurrence-matrix-from-16-bit-image
(why can't we have 16 bit type for the coocurrence matrix computation)

* https://www.sciencedirect.com/topics/engineering/cooccurrence-matrix (simple explanation GLCM)

### Haar-Wavelet with GLCM
Appl. Sci. 2019, 9, 5388; doi:10.3390/app9245388

Method: apply the feature extraction with the glcm to each haar wavelet decomposition result

*Because of their small size and their high degree of localization, microcalcifications represent high-spatial frequencies in the image. The wavelet transform is an attractive option for the detection of high-spatial-frequency components of an image because it can spatially localize high-frequency components. Hence, it was used by many authors for the segmentation of microcalcifications. 15–27*
Source: DOI: 10.1118/1.3121511


In [36]:
def featuresExtractionHaarWavelet(matrix, candidates, features, mask, groundTruth, image, folder):

  # no candidates, no extraction needed
  if (len(candidates) == 0):
    return []

  # angles
  angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

  flag = True
  for index, candidate in enumerate(progress_bar(candidates)):
    
    distances = [1]# probably we need bigger

    # to use them as coordinates they have to be integers
    candidate = candidate.astype(np.int64)

    # candidates are y,x and sigma
    # tolerance to the window
    n = 3
    left = int((candidate[0] - candidate[2] - n) if ((candidate[0] - candidate[2] - n) > 0) else 0)
    right = int((candidate[0] + candidate[2] + n) if ((candidate[0] + candidate[2] + n) < matrix.shape[0]) else matrix.shape[0])
    top = int((candidate[1] - candidate[2] - n) if ((candidate[1] - candidate[2] - n) > 0) else 0)
    bottom = int((candidate[1] + candidate[2] + n) if ((candidate[1] + candidate[2] + n) < matrix.shape[1]) else matrix.shape[1])

    patchCandidate = matrix[left: right,
                            top : bottom]

    # graycomatrix, glcm, receive unsigned integer type
    patchCandidate = patchCandidate.astype(np.uint8)

    #Wavelet Transform

    ## haar wavelet 
    #Apply GLCM on LH, HL, HH

    coeffs2 = pywt.dwt2(patchCandidate, 'haar')

    # low-low, low-high, high-low, high-high
    (LL, (LH, HL, HH)) = coeffs2

    # LL just give information about the details not the edges so unnecessary

    LH = LH.astype(np.uint8)
    HL = HL.astype(np.uint8)
    HH = HH.astype(np.uint8)

    haarDecomposes = [LH, HL, HH]

    for i, haarDecompose in enumerate(haarDecomposes):

      dictFeatures = {}
      dictFeatures = {'name': 'patch_' + str(index) + '_haar_' + str(i) + '_surf_' + str(image.split(".")[0]),
                      'label': patchGroundTruth(candidate, groundTruth)}

      # combination of distances and angles as couples of values
      distAngle = list(itertools.product(distances, angles))

      for distanceAngle in distAngle:

        distance = distanceAngle[0]
        angle = distanceAngle[1]

        # get the degree to use it as name for the column
        name = str(angle*(180.0/np.pi))

        # input image, distance in pixels, angles
        glcm = feature.graycomatrix(haarDecompose, [ distance ], [ angle ])

        # properties
        dictFeatures['contrast'+ str(distance) + name] = feature.graycoprops(glcm, 'contrast')[0][0]
        dictFeatures['dissimilarity' + str(distance) + name] = feature.graycoprops(glcm, 'dissimilarity')[0][0]
        dictFeatures['homogeneity' + str(distance) + name] = feature.graycoprops(glcm, 'homogeneity')[0][0]
        dictFeatures['energy' + str(distance) + name] = feature.graycoprops(glcm, 'energy')[0][0]
        dictFeatures['correlation' + str(distance) + name] = feature.graycoprops(glcm, 'correlation')[0][0]
        dictFeatures['ASM' + str(distance) + name] = feature.graycoprops(glcm, 'ASM')[0][0]

      # add to the dataframe the features for this patch
      features = features.append(dictFeatures, ignore_index=True)

  # save in the csv
  writeFeatures(features, flag, folder, image, 'haar_glcm')

  return features

In [37]:
def writeFeatures(features, flag, folder, image, name):
  if(flag):
    features.to_csv(os.path.join('/content',
                                 'drive',
                                 'MyDrive',
                                 'Results',
                                 folder,
                                 str(image.split('.')[0]) + '_'+ name + '.csv'),
                    mode='a',
                    index=False)
    flag = False
  else:
    features.to_csv(os.path.join('/content',
                                 'drive',
                                 'MyDrive',
                                 'Results',
                                 folder,
                                 str(image.split('.')[0]) + '_'+ name + '.csv'),
                  mode='a',
                  header=False,
                  index=False)
  return flag

# Connected Components

In [38]:
# function to get connected components of the ground truth binary image
def componentsStatsGroundTruth(matrix):
    # getting the info of the components in the ground truth
    # second value is connectivity 4 or 8
    connectedComponentsGroundTruth = cv2.connectedComponentsWithStats(matrix, 8, cv2.CV_32S)

    # Get the results
    # The first cell is the number of labels
    num_labels = connectedComponentsGroundTruth[0]
    # The second cell is the label matrix
    labels = connectedComponentsGroundTruth[1]
    # The third cell is the stat matrix
    stats = connectedComponentsGroundTruth[2]
    # The fourth cell is the centroid matrix
    centroids = connectedComponentsGroundTruth[3]

    return num_labels, labels, stats, centroids

# Show Images

In [39]:
from matplotlib.patches import Circle
import matplotlib.patches as mpatches

# function to draw the grid to display
def display_grid(figure, axis, img, imgGroundTruth, preprocessed, candidates, features):
  # draw in the axis the img
  axis[0][0].imshow(img)
  # switch off the axis of the plot
  axis[0][0].axis('off')
  # set a title for the plot
  axis[0][0].set_title('Image')

  axis[0][1].imshow(imgGroundTruth, cmap='gray')
  axis[0][1].axis('off')
  axis[0][1].set_title('Ground Truth')

  axis[0][2].imshow(imgMask)
  axis[0][2].axis('off')
  axis[0][2].set_title('Breast Mask')

  axis[1][0].imshow(preprocessed, cmap='gray')
  axis[1][0].axis('off')
  axis[1][0].set_title('Preprocessed')

  # draw candidates as circles
  axis[1][1].imshow(preprocessed, cmap='gray')
  axis[1][1].axis('off')
  axis[1][1].set_title('Candidates')

  # Now, loop through coord arrays, and create a circle at each x,y pair
  for y,x,sigma in candidates:

    # blue circle on the candidate
    blob = Circle((x,y), sigma*5, color='blue', fill=False)
    axis[1][1].add_patch(blob)

    # red squares in the patches extracted
    rect=mpatches.Rectangle((x,y),sigma,sigma, 
                        fill=False,
                        color="red",
                       linewidth=2)
    axis[1][1].add_patch(rect)

  axis[1][2].imshow(imgGroundTruth, cmap='gray')
  axis[1][2].axis('off')
  axis[1][2].set_title('compare with ground truth and candidates')

  # Now, loop through coord arrays, and create a circle at each x,y pair
  for y,x,sigma in candidates:
    blob = Circle((x,y), sigma, color='blue', fill=False)
    axis[1][2].add_patch(blob)
  
  return figure, axis

# Main

In [ ]:
import copy
import pandas as pd

#go through the image files 
for image, breastMask, groundTruth in zip(progress_bar(images), breastMasks, groundTruths):

  # to save the features generated with the glcm
  features = pd.DataFrame(dtype=np.float64)

  # 20588020, 7717, 5328, 3787, 5725, 3859, 6934, 50995872
  digits = '5328'

  #if ((digits in image) and (digits in breastMask) and ('mask' in breastMask)):
  if ('mask' in breastMask):
  #if image not in already:

    #upload images
    img = cv2.imread(os.path.join(DATA_DIR,'images',image))
    imgMask = cv2.imread(os.path.join(DATA_DIR, 'masks', breastMask))
    imgGroundTruth = cv2.imread(os.path.join(DATA_DIR, 'groundtruths', image), cv2.IMREAD_GRAYSCALE)
    imgCopy = copy.deepcopy(img)

    # preprocessing # 

    preprocessed = deHazingDarkChannelPriorPy(imgCopy, imgMask)
    preprocessedDil = imgDilation(preprocessed)

    # candidate extraction #

    copyPreprocessedDil = copy.deepcopy(preprocessedDil)
    copyPreprocessedDil = copyPreprocessedDil.astype(np.float32)
    copyPreprocessedDil = cv2.cvtColor(copyPreprocessedDil, cv2.COLOR_BGR2GRAY)
    candidates = candidateExtraction(copyPreprocessedDil, imgMask)

    # feature extraction #

    copyPreprocessed = copy.deepcopy(preprocessed)
    copyPreprocessed = copyPreprocessed.astype(np.float32)
    copyPreprocessed = cv2.cvtColor(copyPreprocessed, cv2.COLOR_BGR2GRAY)
    features = featuresExtractionHaarWavelet(copyPreprocessed, candidates, features, imgMask, imgGroundTruth, image, 'dehazingDC_GF+Dilation+DoH+(Haar+GLCM)')

    # ML must be applied for the classification of the features extracted

    ################ ERASE MEMORY
    # import gc
    # del features
    # del preprocessed
    # del candidates
    # # del blobs
    # del copyPreprocessed
    # del imgCopy
    # del img
    # del imgMask
    # del imgGroundTruth
    # gc.collect()
    ############################

    # end image processing part #

    # display related #

    # matrix of plots and size of the figure
    # figure, axis = plt.subplots(2, 3, figsize=(15,15))
    # display_grid(figure, axis, img, imgGroundTruth, preprocessed, candidates, features)
    # plt.subplots_adjust(wspace=0, hspace=0)

    # # # display figure with image
    # plt.show() 

## CONCLUSIONS FOR PREPROCESSING

* still missing quantum noise supression

* details in the phd defense file

* Observations from the results:
    * fiber intersections may also appear as bright spots (false positives)

* THINGS WE NOTICE BETWEEN BOTH DEHAZING METHODS
    * Better suppression of fatty tissue (noise) and greater enhancement of brightness of desired feature (microcalcifications)
    * sometimes for the other dehazing method black patches become present in the fatty tissue
    * this did not happen in the dehazing with dark channel prior (and guided filter)
    * sharper
    * enhances the contrast and details

* Observations from the results:
    * images with pectoral muscule cause false positives